# Data processing

In [2]:
import os, sys
import random

In [3]:
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")

In [4]:
os.environ["HF_DATASETS_CACHE"] = "/scratch/work/koutchc1/cache/huggingface/datasets/"

In [5]:
import pandas as pd

## Loading all data 

In [6]:
path = "/scratch/work/koutchc1/datasets/falconcode"
runs_df = pd.read_csv(os.path.join(path, "falconcode_v1_table_runs.csv"))
problems_df = pd.read_csv(os.path.join(path, "falconcode_v1_table_problems.csv"))
courses_df = pd.read_csv(os.path.join(path, "falconcode_v1_table_courses.csv"))
samples_df = pd.read_csv(os.path.join(path, "falconcode_v1_code_samples.csv"))

In [13]:
print(problems_df["testcase"].iloc[65])

from cs110 import autograder
import random, math, py_compile

# Runs the Python script and sees if it passes the test(s)
def test_passed():
    try:       
        py_compile.compile("lsn23_soundboard.py", doraise=True)
        print("Thank you for your submission. Your instructor will let you know if there is a problem.")
        return 100.0
    except:
        print("There appears to be a syntax error in your code.")
        return 0.0
    
        
# Runs your code in an IDE (for testing purposes)
if __name__ == '__main__':    
    result = test_passed()
    print("Unit Test Returned:", result)
















In [7]:
runs_df.course_id

0          3
1          3
2          3
3          3
4          3
          ..
1433318    2
1433319    2
1433320    2
1433321    2
1433322    2
Name: course_id, Length: 1433323, dtype: int64

In [8]:
courses_df

,id,name,semester,year
0,2,CS110,Spring,2021
1,3,CS110,Fall,2021
2,4,CS110,Spring,2022


### Processing the problems dataframe to have a representation of the skills as a string

In [30]:
from collections import defaultdict

def map_problems_to_concepts(pdf):
    """ 
    Obtain a dictionary which, for each course,
    map each problem to the set of concepts it's associated to
    """
    problem_to_concepts = defaultdict(set)
    cpdf = pdf.set_index(["course_id", "id"])
    
    # Multi-skill separator: If there are multiple skills in a question, 
    # we separate the skills with an underline _
    
    # Character replacement: If there is an underline _ in the question 
    # and skill of original data, replace it with ####. 

    cpdf = cpdf.iloc[:, 5:].to_dict("index")
    for (_, prob_id), record in cpdf.items():
        concepts = [k.replace("_", "####") for k, v in record.items() if v]
        prob_id = prob_id.replace("_", "####")
        problem_to_concepts[prob_id].update(concepts)

    return problem_to_concepts

In [37]:
problems_df["id"] = [prob_id.replace("_", "####") for prob_id in problems_df["id"]]
runs_df["problem_id"] = [prob_id.replace("_", "####") for prob_id in runs_df["problem_id"]]

In [38]:
problem_to_concepts = map_problems_to_concepts(problems_df)
problem_to_concepts

defaultdict(set,
            {'a3####3####animals': {'concept####list',
              'list',
              'loop####elements',
              'output'},
             'a3####1####cargo': {'concept####list',
              'input####cast',
              'list',
              'output'},
             'a3####3####games': {'concept####list',
              'loop####elements',
              'output'},
             'a3####4####heights': {'concept####list',
              'function####call',
              'output'},
             'pex3': {'assignment',
              'concept####list',
              'conditional',
              'function####call',
              'function####def',
              'function####return',
              'list',
              'list####2d',
              'loop####elements',
              'loop####nested',
              'loop####until',
              'output'},
             'pex1': {'assignment',
              'concept####list',
              'conditional',
              'inpu

In [39]:
problems_df["concept_list"] = ["_".join(problem_to_concepts[pid]) 
                               for pid in problems_df["id"]]

### Processing: Merging all data into a single table for easier processing later 

In [40]:
merged_df = pd.merge(runs_df, problems_df, 
                     how="left",
                     left_on=["problem_id", "course_id"], 
                     right_on=["id", "course_id"], 
                     suffixes=("", "_p"),
                     validate="many_to_one", sort=False)
merged_df = pd.merge(merged_df, courses_df, 
                     how="left",
                     left_on="course_id", right_on="id", 
                     suffixes=("", "_c"),
                     validate="many_to_one", sort=False)
merged_df = pd.merge(merged_df, samples_df, 
                     how="left",
                     left_on="code_hash", 
                     right_on="hash", 
                     suffixes=("", "_s"),
                     validate="many_to_one", sort=False)
merged_df

,id,student_id,course_id,problem_id,timestamp,score,code_hash,id_p,type,exam,...,item_set,tuple,concept_list,id_c,name,semester,year,hash,source_code,redacted
0,1,6841f916-e8eb-40a8-ad96-486f030ebb96,3,Airstrike,2021-12-29 02:11:55,-1,4056437beff8be1bd548271a2c7b79e334adb832,Airstrike,project,0.0,...,0.0,0.0,concept####list_list####2d_loop####elements_co...,3.0,CS110,Fall,2021.0,4056437beff8be1bd548271a2c7b79e334adb832,# Project Part 3 - Airstrike\n# REDACTED_NAME(...,1.0
1,2,35a78ee4-f99e-4914-a38c-fea25a3e1e8d,3,Airstrike,2021-12-28 15:16:43,-1,6965122b550742f95679bb64a9974af36b2e2e73,Airstrike,project,0.0,...,0.0,0.0,concept####list_list####2d_loop####elements_co...,3.0,CS110,Fall,2021.0,6965122b550742f95679bb64a9974af36b2e2e73,# Project Part 3 - Airstrike\n# REDACTED_NAME(...,1.0
2,3,8e6da047-045a-4be3-ac88-f08854f793ef,3,Recon,2021-12-24 23:02:51,-1,3484f44d0d9fa81b2e73adbca37cef65a6a1e06e,Recon,project,0.0,...,0.0,0.0,concept####list_list####2d_loop####elements_co...,3.0,CS110,Fall,2021.0,3484f44d0d9fa81b2e73adbca37cef65a6a1e06e,# Project Part 2 - Recon\n# REDACTED_NAME(S)\n...,1.0
3,4,8e6da047-045a-4be3-ac88-f08854f793ef,3,Recon,2021-12-24 23:02:47,0,3484f44d0d9fa81b2e73adbca37cef65a6a1e06e,Recon,project,0.0,...,0.0,0.0,concept####list_list####2d_loop####elements_co...,3.0,CS110,Fall,2021.0,3484f44d0d9fa81b2e73adbca37cef65a6a1e06e,# Project Part 2 - Recon\n# REDACTED_NAME(S)\n...,1.0
4,5,3387d072-70ef-47b4-96b3-29b7440314f7,3,Airstrike,2021-12-24 04:06:39,-1,f13191146e17267a68ec072fc5c138150c475b32,Airstrike,project,0.0,...,0.0,0.0,concept####list_list####2d_loop####elements_co...,3.0,CS110,Fall,2021.0,f13191146e17267a68ec072fc5c138150c475b32,# Project Part 3 - Airstrike\n# REDACTED_NAME(...,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1433318,1433319,8a9837cb-a545-4eaa-90b0-d7e37caeadb2,2,a2####4####colon,2021-07-22 22:53:24,100,1062dfa20789add3b1329902b29b8f2e4bd5f5d7,a2####4####colon,skill,1.0,...,0.0,0.0,function####call_output_concept####list_list,2.0,CS110,Spring,2021.0,1062dfa20789add3b1329902b29b8f2e4bd5f5d7,# AUTOGRADER IMPORT REMOVED\n\n# -------------...,0.0
1433319,1433320,8a9837cb-a545-4eaa-90b0-d7e37caeadb2,2,a2####6####combat,2021-07-22 22:56:09,0,f7388d8a008482c9c8bcb211ee7edeb795978a25,a2####6####combat,lab,1.0,...,0.0,0.0,concept####list_conditional_output_assignment_...,2.0,CS110,Spring,2021.0,f7388d8a008482c9c8bcb211ee7edeb795978a25,# AUTOGRADER IMPORT REMOVED\n\n# -------------...,0.0
1433320,1433321,8a9837cb-a545-4eaa-90b0-d7e37caeadb2,2,a2####6####combat,2021-07-22 22:56:27,50,0047289b08bbd4413f5d8feb52b0cf48d9a19ac9,a2####6####combat,lab,1.0,...,0.0,0.0,concept####list_conditional_output_assignment_...,2.0,CS110,Spring,2021.0,0047289b08bbd4413f5d8feb52b0cf48d9a19ac9,# AUTOGRADER IMPORT REMOVED\n\n# -------------...,0.0
1433321,1433322,8a9837cb-a545-4eaa-90b0-d7e37caeadb2,2,a2####6####combat,2021-07-22 22:56:57,100,d227c3ba4048217bed20787e54e3c2ede8aee929,a2####6####combat,lab,1.0,...,0.0,0.0,concept####list_conditional_output_assignment_...,2.0,CS110,Spring,2021.0,d227c3ba4048217bed20787e54e3c2ede8aee929,# AUTOGRADER IMPORT REMOVED\n\n# -------------...,0.0


In [41]:
columns = ["id", "student_id", "problem_id", "exam", "timestamp", "course_id", "score", "max_score",
           "code_hash", "source_code",  "concept_list"]
merged_df = merged_df[columns]

In [42]:
merged_df.to_csv(os.path.join(path, "falconcode_v1_merged.csv"))

In [43]:
merged_df[merged_df.max_score == 100].course_id.unique()

array([3, 2, 4])

In [44]:
merged_df[merged_df.course_id == 3]

,id,student_id,problem_id,exam,timestamp,course_id,score,max_score,code_hash,source_code,concept_list
0,1,6841f916-e8eb-40a8-ad96-486f030ebb96,Airstrike,0.0,2021-12-29 02:11:55,3,-1,0.0,4056437beff8be1bd548271a2c7b79e334adb832,# Project Part 3 - Airstrike\n# REDACTED_NAME(...,concept####list_list####2d_loop####elements_co...
1,2,35a78ee4-f99e-4914-a38c-fea25a3e1e8d,Airstrike,0.0,2021-12-28 15:16:43,3,-1,0.0,6965122b550742f95679bb64a9974af36b2e2e73,# Project Part 3 - Airstrike\n# REDACTED_NAME(...,concept####list_list####2d_loop####elements_co...
2,3,8e6da047-045a-4be3-ac88-f08854f793ef,Recon,0.0,2021-12-24 23:02:51,3,-1,0.0,3484f44d0d9fa81b2e73adbca37cef65a6a1e06e,# Project Part 2 - Recon\n# REDACTED_NAME(S)\n...,concept####list_list####2d_loop####elements_co...
3,4,8e6da047-045a-4be3-ac88-f08854f793ef,Recon,0.0,2021-12-24 23:02:47,3,0,0.0,3484f44d0d9fa81b2e73adbca37cef65a6a1e06e,# Project Part 2 - Recon\n# REDACTED_NAME(S)\n...,concept####list_list####2d_loop####elements_co...
4,5,3387d072-70ef-47b4-96b3-29b7440314f7,Airstrike,0.0,2021-12-24 04:06:39,3,-1,0.0,f13191146e17267a68ec072fc5c138150c475b32,# Project Part 3 - Airstrike\n# REDACTED_NAME(...,concept####list_list####2d_loop####elements_co...
...,...,...,...,...,...,...,...,...,...,...,...
620735,620736,7dd9b46c-48a7-4efc-a1dd-df969911a6db,lsn19####skill1,0.0,2021-10-19 18:13:39,3,100,100.0,a96ef05ba4fa6fbcc834a645d427a1ad62240068,# AUTOGRADER IMPORT REMOVED\n\n# -------------...,concept####list_list####2d_output_assignment_l...
620736,620737,7dd9b46c-48a7-4efc-a1dd-df969911a6db,lsn19####skill2,0.0,2021-10-19 18:15:54,3,100,100.0,97f6eb1ad4321481d0eabc86640c10cee2a2c490,# AUTOGRADER IMPORT REMOVED\n\n# -------------...,concept####list_list####2d
620737,620738,155f7e71-2d0c-4b63-b56b-6924906aa3f6,lsn5####projectile,NaN,2021-10-19 18:18:19,3,0,NaN,85b014659ca03a7235094917c4adaa49a319adde,# AUTOGRADER IMPORT REMOVED\nimport math\n\n# ...,NaN
620738,620739,155f7e71-2d0c-4b63-b56b-6924906aa3f6,lsn5####projectile,NaN,2021-10-19 18:18:30,3,0,NaN,5b18a94f10f807565e9ff07d661404666380fed8,# AUTOGRADER IMPORT REMOVED\nimport math\n\n# ...,NaN
